In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import os, pandas as pd, torch, torchvision
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from PIL import Image
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
img_root = "/content/drive/MyDrive/GeoSentioMap/data"
csv_path = "/content/drive/MyDrive/GeoSentioMap/metadata.csv"

# Emotion Mapping
emotion_map = {'peaceful': 0, 'neutral': 1, 'energetic': 2, 'chaotic': 3}
num_classes = len(emotion_map)

# Image Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

# Metadata encoder
def encode_meta(row):
    location_map = {'kerala': 0, 'chennai': 1}
    weather_map = {'sunny': 0, 'rainy': 1, 'cloudy': 2}
    time_map = {'morning': 0, 'afternoon': 1, 'evening': 2}
    return [
        location_map.get(row['location'], 0),
        weather_map.get(row['weather'], 0),
        time_map.get(row['time_of_day'], 0)
    ]

# Custom Dataset
class EmotionDataset(Dataset):
    def __init__(self, df, img_root):
        self.df = df
        self.img_root = img_root

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_root, row['location'], row['filename'])

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            image = Image.new("RGB", (224, 224), color="white")

        image = transform(image)
        meta = torch.tensor(encode_meta(row), dtype=torch.float32)
        label = torch.tensor(emotion_map[row['emotion_label']], dtype=torch.long)
        return image, meta, label

# Load CSV
df = pd.read_csv(csv_path)
dataset = EmotionDataset(df, img_root)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

#  Dynamic class weight fix (based on available labels only)
labels = df['emotion_label'].map(emotion_map).dropna().astype(int)
unique_labels = np.unique(labels)
class_weights = compute_class_weight(class_weight='balanced', classes=unique_labels, y=labels)
weights_tensor = torch.ones(num_classes, dtype=torch.float32)
weights_tensor[unique_labels] = torch.tensor(class_weights, dtype=torch.float32)
weights_tensor = weights_tensor.to(device)

# Load ResNet
resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
for param in resnet.parameters():
    param.requires_grad = False
for param in resnet.layer4.parameters():
    param.requires_grad = True
resnet.fc = nn.Identity()

# Final Model
class EmotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = resnet
        self.meta_fc = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 + 16, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, img, meta):
        img_feat = self.cnn(img)
        meta_feat = self.meta_fc(meta)
        combined = torch.cat([img_feat, meta_feat], dim=1)
        return self.classifier(combined)

# Initialize model
model = EmotionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(weight=weights_tensor)

# 🔁 Training
for epoch in range(20):
    model.train()
    total_loss = 0
    for imgs, metas, labels in loader:
        imgs = imgs.to(device)
        metas = metas.to(device)
        labels = labels.to(device)

        preds = model(imgs, metas)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f" Epoch {epoch+1} | Loss: {total_loss:.4f}")

# Save model
save_path = "/content/drive/MyDrive/GeoSentioMap/emotion_model_final.pt"
torch.save(model.state_dict(), save_path)
print(f" Final Model saved at: {save_path}")


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Epoch 1 | Loss: 18.9345


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Epoch 2 | Loss: 15.7387


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

✅ Epoch 18 | Loss: 0.9041


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Epoch 19 | Loss: 1.0061


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Epoch 20 | Loss: 0.9229
✅ Final Model saved at: /content/drive/MyDrive/GeoSentioMap/emotion_model_final.pt


In [16]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import os

# --- 1. Device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 2. Emotion Map ---
emotion_map = {0: 'peaceful', 1: 'neutral', 2: 'energetic', 3: 'chaotic'}

# --- 3. Transforms (same as training) ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# --- 4. Metadata encoding ---
def encode_meta(location, weather, time_of_day):
    location_map = {'kerala': 0, 'chennai': 1}
    weather_map = {'sunny': 0, 'rainy': 1, 'cloudy': 2}
    time_map = {'morning': 0, 'afternoon': 1, 'evening': 2}
    return torch.tensor([
        location_map.get(location, 0),
        weather_map.get(weather, 0),
        time_map.get(time_of_day, 0)
    ], dtype=torch.float32).to(device)

# --- 5. Define model (same as training) ---
class EmotionNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        for param in resnet.parameters():
            param.requires_grad = False
        for param in resnet.layer4.parameters():
            param.requires_grad = True
        resnet.fc = nn.Identity()

        self.cnn = resnet
        self.meta_fc = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 + 16, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 4)
        )

    def forward(self, img, meta):
        img_feat = self.cnn(img)
        meta_feat = self.meta_fc(meta)
        combined = torch.cat([img_feat, meta_feat], dim=1)
        return self.classifier(combined)

# --- 6. Load model ---
model = EmotionNet().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/GeoSentioMap/emotion_model_final.pt"))
model.eval()

# --- 7. Load your test image ---
image_path = "/content/drive/MyDrive/GeoSentioMap/test4.webp"  #  CHANGE THIS
img = Image.open(image_path).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(device)  # shape: [1, 3, 224, 224]

# --- 8. Create metadata ( adjust values) ---
meta_tensor = encode_meta("kerala", "sunny", "morning").unsqueeze(0)  # shape: [1, 3]

# --- 9. Predict ---
with torch.no_grad():
    output = model(img_tensor, meta_tensor)
    pred = torch.argmax(output, dim=1).item()
    print(f" Predicted Emotion: {emotion_map[pred]}")


🧠 Predicted Emotion: peaceful
